Instructions: 

--- Day 7: Handy Haversacks ---

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

    A bright white bag, which can hold your shiny gold bag directly.
    A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
    A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
    A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)


In [71]:
puzzle_input = []

with open('puzzle_input') as infile:
    for line in infile: 
        puzzle_input.append(line.rstrip('\n'))

In [2]:
sample_puzzle_input = [
    'light red bags contain 1 bright white bag, 2 muted yellow bags.',
'dark orange bags contain 3 bright white bags, 4 muted yellow bags.',
'bright white bags contain 1 shiny gold bag.',
'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.',
'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.',
'dark olive bags contain 3 faded blue bags, 4 dotted black bags.',
'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.',
'faded blue bags contain no other bags.',
'dotted black bags contain no other bags.'
]

In [33]:
import re

def extract_bag_dict_entry(raw_string):
    
    # first extract the bag in question (main bag)
    main_bag_search = re.search('(.+ .+) bags contain ', raw_string)
    
    main_bag = main_bag_search.group(1)
    
    # extract the other bit of info
    contents_raw = re.sub('.+ .+ bags contain ', '', raw_string)
    
    contents_match = re.findall('(\d+) (.+? .+?) bag', contents_raw)
        
    bag_contents_dict = {}

    for contents_description in contents_match:

        bag_contents_dict[contents_description[1]] = int(contents_description[0])

    return {main_bag: bag_contents_dict}

In [35]:
print(extract_bag_dict_entry(sample_puzzle_input[0]))

{'light red': {'bright white': 1, 'muted yellow': 2}}


In [43]:
test_dict = extract_bag_dict_entry(sample_puzzle_input[0])
print(list(test_dict.keys())[0])

light red


In [54]:
def create_bag_dict(puzzle_input):
    
    combined_bag_dict = {}
    
    for line in puzzle_input:
        
        current_bag_dict = extract_bag_dict_entry(line)
        
        # combine the two dictionaries using this nifty line
        combined_bag_dict = {**combined_bag_dict, **current_bag_dict}
        
    return combined_bag_dict

In [57]:
bag_dict = create_bag_dict(sample_puzzle_input)
print(bag_dict)

{'light red': {'bright white': 1, 'muted yellow': 2}, 'dark orange': {'bright white': 3, 'muted yellow': 4}, 'bright white': {'shiny gold': 1}, 'muted yellow': {'shiny gold': 2, 'faded blue': 9}, 'shiny gold': {'dark olive': 1, 'vibrant plum': 2}, 'dark olive': {'faded blue': 3, 'dotted black': 4}, 'vibrant plum': {'faded blue': 5, 'dotted black': 6}, 'faded blue': {}, 'dotted black': {}}


In [56]:
def check_if_bag_can_fit(bag_to_fit, bigger_bag, bag_dict):
    
    # if the bag fits directly into the bag, it's a hit (so True)
    if bag_to_fit in bag_dict[bigger_bag]:
        
        return True
    
    # if the bag can't fit anything in, it's a miss (so False)
    elif not bag_dict[bigger_bag]:
        
        return False
    
    # if the bag doesn't fit directly, but can fit other bags, look at those:
    else:
        
        return any([check_if_bag_can_fit(bag_to_fit, bag, bag_dict) for bag in bag_dict[bigger_bag]])       
        

In [65]:
print(check_if_bag_can_fit('shiny gold', 'vibrant plum', bag_dict))

False


In [68]:
def count_bag_fits(bag_to_fit, puzzle_input):
    
    bag_dict = create_bag_dict(puzzle_input)
    
    counter = 0
    
    for bigger_bag in bag_dict:
        
        if check_if_bag_can_fit(bag_to_fit, bigger_bag, bag_dict):
            
            counter += 1
            
    return counter

In [72]:
count_bag_fits('shiny gold', puzzle_input)

126

--- Part Two ---

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

    faded blue bags contain 0 other bags.
    dotted black bags contain 0 other bags.
    vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
    dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?


In [73]:
sample_puzzle_input_2 = [
    'shiny gold bags contain 2 dark red bags.',
'dark red bags contain 2 dark orange bags.',
'dark orange bags contain 2 dark yellow bags.',
'dark yellow bags contain 2 dark green bags.',
'dark green bags contain 2 dark blue bags.',
'dark blue bags contain 2 dark violet bags.',
'dark violet bags contain no other bags.'
]

In [74]:
bag_dict

{'light red': {'bright white': 1, 'muted yellow': 2},
 'dark orange': {'bright white': 3, 'muted yellow': 4},
 'bright white': {'shiny gold': 1},
 'muted yellow': {'shiny gold': 2, 'faded blue': 9},
 'shiny gold': {'dark olive': 1, 'vibrant plum': 2},
 'dark olive': {'faded blue': 3, 'dotted black': 4},
 'vibrant plum': {'faded blue': 5, 'dotted black': 6},
 'faded blue': {},
 'dotted black': {}}

In [79]:
sum(bag_dict['faded blue'].values())

0

In [88]:
def count_total_inner_bags(outer_bag, bag_dict, count=0):
    
    count = sum(bag_dict[outer_bag].values())
    
    # if the bag can contain other bags, add on how many bags should go into that bag
    if bag_dict[outer_bag]:
        
        for inner_bag in bag_dict[outer_bag]:
            
            count += (count_total_inner_bags(inner_bag, bag_dict, count=count)) * bag_dict[outer_bag][inner_bag]
            
    return count

In [90]:
def part_2_puzzle_answer(outer_bag, puzzle_input):
    
    bag_dict = create_bag_dict(puzzle_input)
    
    return count_total_inner_bags(outer_bag, bag_dict)

In [91]:
print(part_2_puzzle_answer('shiny gold', sample_puzzle_input))

32


In [92]:
print(part_2_puzzle_answer('shiny gold', sample_puzzle_input_2))

126


In [93]:
print(part_2_puzzle_answer('shiny gold', puzzle_input))

220149
